In [1]:
from pyspark.sql import SparkSession

from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession \
    .builder \
    .appName("crear_one_big_table") \
    .enableHiveSupport() \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2023-11-19T16:32:39,867 WARN [Thread-4] org.apache.hadoop.util.NativeCodeLoader - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2023-11-19T16:32:40,599 WARN [Thread-4] org.apache.spark.util.Utils - Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
df_bigtable = spark.sql("""
    select 
        ae.id_sensor as act_eve_id_sensor, 
        ae.pulse_rate as act_eve_pulse_rate,
        ae.id_actividad as act_eve_id_actividad,
        ae.longitud as act_eve_longitud,
        ae.latitud as act_eve_latitud,
        ae.fecha_hora as act_eve_fecha_hora, 
        a.id_usuario as act_id_usuario, 
        a.id_dispositivo as act_id_dispositivo,
        a.id_plan as act_id_plan,
        a.tipo_actividad as act_tipo_actividad,
        a.pais as act_pais,
        a.longitud as act_longitud,
        a.latitud as act_latitud, 
        a.duracion as act_duracion,
        a.fecha_hora as act_fecha_hora,
        u.nombre as usu_nombre, 
        u.apellido as usu_apellido,
        u.email as usu_email,
        u.sexo as usu_sexo,
        u.pais as usu_pais, 
        u.actividad as usu_actividad,
        u.subscripcion as usu_subscripcion,
        u.fecha_subs as usu_fecha_subs,
        u.fecha_renov as usu_fecha_renov,
        u.ultimo_pago as usu_ultimo_pago,
        d.model as dis_model,
        d.brand as dis_brand,
        d.operating_system as dis_operating_system,
        pe.tipo as pla_tipo,
        pe.plan_duracion as pla_plan_duracion,
        pe.instrucciones as pla_instrucciones, 
        pe.objetivo as pla_objetivo        
    from 
        datalake.actividades_eventos ae 
            INNER JOIN datalake.actividades a ON (ae.id_actividad = a.id_actividad)
            INNER JOIN datalake.usuarios u ON (a.id_usuario = u.id_usuario) 
            INNER JOIN datalake.dispositivos d ON (a.id_dispositivo = d.id_dispositivo)
            INNER JOIN datalake.planes_de_entrenamiento pe ON (a.id_plan = pe.id_plan and pe.id_usuario = a.id_usuario)
""")

2023-11-19T16:32:42,772 INFO [Thread-4] org.apache.hadoop.hive.conf.HiveConf - Found configuration file file:/home/ort/spark/conf/hive-site.xml
2023-11-19T16:32:42,917 WARN [Thread-4] org.apache.hadoop.hive.conf.HiveConf - HiveConf of name hive.metastore.wm.default.pool.size does not exist
2023-11-19T16:32:42,917 WARN [Thread-4] org.apache.hadoop.hive.conf.HiveConf - HiveConf of name hive.llap.task.scheduler.preempt.independent does not exist
2023-11-19T16:32:42,917 WARN [Thread-4] org.apache.hadoop.hive.conf.HiveConf - HiveConf of name hive.llap.output.format.arrow does not exist
2023-11-19T16:32:42,917 WARN [Thread-4] org.apache.hadoop.hive.conf.HiveConf - HiveConf of name hive.tez.llap.min.reducer.per.executor does not exist
2023-11-19T16:32:42,917 WARN [Thread-4] org.apache.hadoop.hive.conf.HiveConf - HiveConf of name hive.arrow.root.allocator.limit does not exist
2023-11-19T16:32:42,917 WARN [Thread-4] org.apache.hadoop.hive.conf.HiveConf - HiveConf of name hive.vectorized.use.che

In [3]:
df_bigtable.printSchema()

root
 |-- act_eve_id_sensor: string (nullable = true)
 |-- act_eve_pulse_rate: integer (nullable = true)
 |-- act_eve_id_actividad: string (nullable = true)
 |-- act_eve_longitud: float (nullable = true)
 |-- act_eve_latitud: float (nullable = true)
 |-- act_eve_fecha_hora: timestamp (nullable = true)
 |-- act_id_usuario: string (nullable = true)
 |-- act_id_dispositivo: string (nullable = true)
 |-- act_id_plan: string (nullable = true)
 |-- act_tipo_actividad: string (nullable = true)
 |-- act_pais: string (nullable = true)
 |-- act_longitud: float (nullable = true)
 |-- act_latitud: float (nullable = true)
 |-- act_duracion: integer (nullable = true)
 |-- act_fecha_hora: timestamp (nullable = true)
 |-- usu_nombre: string (nullable = true)
 |-- usu_apellido: string (nullable = true)
 |-- usu_email: string (nullable = true)
 |-- usu_sexo: string (nullable = true)
 |-- usu_pais: string (nullable = true)
 |-- usu_actividad: string (nullable = true)
 |-- usu_subscripcion: string (nullab

In [4]:
df_bigtable.write.mode("overwrite").parquet("/datalake/curated/bigtable_data_fitness")

2023-11-19T16:32:49,112 WARN [Thread-4] org.apache.spark.sql.catalyst.util.package - Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [6]:
spark.sql("""
CREATE EXTERNAL TABLE IF NOT EXISTS datalake.bigtable_data_fitness (
    act_eve_id_sensor STRING,
    act_eve_pulse_rate INT,
    act_eve_id_actividad STRING,
    act_eve_longitud FLOAT,
    act_eve_latitud FLOAT,
    act_eve_fecha_hora TIMESTAMP,
    act_id_usuario STRING,
    act_id_dispositivo STRING,
    act_id_plan STRING,
    act_tipo_actividad STRING,
    act_pais STRING,
    act_longitud FLOAT,
    act_latitud FLOAT,
    act_duracion INT,
    act_fecha_hora TIMESTAMP,
    usu_nombre STRING,
    usu_apellido STRING,
    usu_email STRING,
    usu_sexo STRING,
    usu_pais STRING,
    usu_actividad STRING,
    usu_subscripcion STRING,
    usu_fecha_subs DATE,
    usu_fecha_renov DATE,
    usu_ultimo_pago DATE,
    dis_model STRING,
    dis_brand STRING,
    dis_operating_system STRING,
    pla_tipo STRING,
    pla_plan_duracion FLOAT,
    pla_instrucciones STRING,
    pla_objetivo STRING
)
STORED AS PARQUET
LOCATION '/datalake/curated/bigtable_data_fitness';
""")



2023-11-19T15:10:40,490 INFO [Thread-4] org.apache.hadoop.hive.ql.security.authorization.plugin.sqlstd.SQLStdHiveAccessController - Created SQLStdHiveAccessController for session context : HiveAuthzSessionContext [sessionString=a2450c7e-11e4-47d5-922d-d4ba01540fbd, clientType=HIVECLI]
2023-11-19T15:10:40,502 WARN [Thread-4] org.apache.hadoop.hive.ql.session.SessionState - METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
2023-11-19T15:10:40,536 INFO [Thread-4] hive.metastore - Mestastore configuration hive.metastore.filter.hook changed from org.apache.hadoop.hive.metastore.DefaultMetaStoreFilterHookImpl to org.apache.hadoop.hive.ql.security.authorization.plugin.AuthorizationMetaStoreFilterHook
2023-11-19T15:10:41,250 INFO [Thread-4] hive.metastore - Closed a connection to metastore, current connections: 0
2023-11-19T15:10:41,252 INFO [Thread-4] hive.metastore - Trying to connect to metastore with URI thrift://l

DataFrame[]

In [7]:
spark.sql("SELECT * FROM datalake.bigtable_data_fitness").show()

+--------------------+------------------+--------------------+----------------+---------------+-------------------+--------------------+--------------------+--------------------+------------------+--------------+------------+-----------+------------+-------------------+-----------+-------------+--------------------+----------+--------------+-------------+----------------+--------------+---------------+---------------+----------------+---------+--------------------+----------------+-----------------+--------------------+--------------------+
|   act_eve_id_sensor|act_eve_pulse_rate|act_eve_id_actividad|act_eve_longitud|act_eve_latitud| act_eve_fecha_hora|      act_id_usuario|  act_id_dispositivo|         act_id_plan|act_tipo_actividad|      act_pais|act_longitud|act_latitud|act_duracion|     act_fecha_hora| usu_nombre| usu_apellido|           usu_email|  usu_sexo|      usu_pais|usu_actividad|usu_subscripcion|usu_fecha_subs|usu_fecha_renov|usu_ultimo_pago|       dis_model|dis_brand|dis_o